In [43]:
import omdb
import sqlite3
import pandas as pd
import os
import requests
import shutil
import io
import PIL.Image as Image

# first read in OMDB python key
with open('api_key') as f:
    lines = f.readlines()
api_key = lines[0]
omdb.set_default('apikey', api_key)


In [2]:
# set acronyms
st = {'tos': {'title': 'The Original Series', 'imdbid': 'tt0060028'},
      'tas': {'title': 'The Animated Series', 'imdbid': 'tt0069637'},
      'tng': {'title': 'The Next Generation', 'imdbid': 'tt0092455'},
      'ds9': {'title': 'Deep Space Nine',     'imdbid': 'tt0106145'},
      'voy': {'title': 'Voyager',             'imdbid': 'tt0112178'},
      'ent': {'title': 'Enterprise',          'imdbid': 'tt0244365'},
      'dis': {'title': 'Discovery',           'imdbid': 'tt5171438'},
      'sho': {'title': 'Short Treks',         'imdbid': 'tt9059594'},
      'pic': {'title': 'Picard',              'imdbid': 'tt8806524'},
      'lds': {'title': 'Lower Decks',         'imdbid': 'tt9184820'},
      'pro': {'title': 'Prodigy',             'imdbid': 'tt9795876'},
      'snw': {'title': 'Strange New Worlds',  'imdbid': 'tt12327578'},
      } 

In [28]:
db_filename = "treklist.db"

# delete current database
os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()
conn.execute("DROP TABLE IF EXISTS series")
conn.execute("CREATE TABLE series (abb TEXT, title TEXT, imdb_id TEXT, year TEXT, total_seasons INTEGER, poster BLOB, rated TEXT)")
for key in st.keys():
    res = omdb.imdbid(st[key]['imdbid'])
    img = requests.get(res['poster'], stream=True)

    cmd = "INSERT INTO series (abb, title, imdb_id, year, total_seasons, poster, rated) VALUES " + \
         f"('{key}', '{res['title']}', '{res['imdb_id']}', '{res['year']}', {res['total_seasons']}, ?, '{res['rated']}');"
    curs.execute(cmd, [sqlite3.Binary(img.content)])
    conn.commit()

    # insert poster
    #cmd = f"UPDATE series SET poster = ? WHERE imdb_id = {res['imdb_id']}"
    #with open('test.jpg','wb') as f:
    #    shutil.copyfileobj(img.raw, f)
    #curs.execute(cmd, )

df = pd.read_sql_query("SELECT * FROM series", conn)
print(df)

    abb                           title     imdb_id       year  total_seasons  \
0   tos                       Star Trek   tt0060028  1966–1969              3   
1   tas  Star Trek: The Animated Series   tt0069637  1973–1975              2   
2   tng  Star Trek: The Next Generation   tt0092455  1987–1994              7   
3   ds9      Star Trek: Deep Space Nine   tt0106145  1993–1999              7   
4   voy              Star Trek: Voyager   tt0112178  1995–2001              7   
5   ent           Star Trek: Enterprise   tt0244365  2001–2005              4   
6   dis            Star Trek: Discovery   tt5171438      2017–              5   
7   sho          Star Trek: Short Treks   tt9059594      2018–              2   
8   pic               Star Trek: Picard   tt8806524      2020–              3   
9   lds          Star Trek: Lower Decks   tt9184820      2020–              3   
10  pro              Star Trek: Prodigy   tt9795876      2021–              1   
11  snw   Star Trek: Strange

In [9]:
for key in st.keys():
    conn.execute(f"DROP TABLE IF EXISTS {key}")
    conn.execute(f"CREATE TABLE {key} (season INTEGER, episode INTEGER, title TEXT)")

In [47]:

curs.execute("SELECT * FROM series WHERE imdb_id = 'tt0106145'")
record = curs.fetchall()
image = Image.open(io.BytesIO(record[0][5]))
image.show()

In [39]:
record[0].keys()
#res = omdb.imdbid(st['tos']['imdbid'])
#print(res)

AttributeError: 'tuple' object has no attribute 'keys'